In [1]:
import numpy as np
import matplotlib.pyplot as plt

from pprint import pprint

import numpy as np
from sklearn.preprocessing import scale
import sklearn.metrics as metrics

import matplotlib.pyplot as plt
import matplotlib.animation as ani

import mlgrad.models as models
import mlgrad.regr as regr
import mlgrad.cls as cls
import mlgrad.pca as pca
import mlgrad.plots as plots
import mlgrad.funcs as funcs
import mlgrad.funcs2 as funcs2
import mlgrad.loss as loss
import mlgrad.gd as gd
import mlgrad.smooth as smooth
import mlgrad.array_transform as at
from mlgrad.af import averaging_function
import mlgrad.avragg as avragg

import rampy
import pybaselines
import random

from irsa import io
from irsa.preprocess import despike, utils
import irsa.widgets as widgets

import scipy.special as special

import copy

import ipywidgets
from IPython.display import display

%matplotlib widget

In [2]:
def plot_zscore_series(key, i, zval):
    plt.figure(figsize=(12,2))
    # plt.title(f"Modified z-score: {key}: {i}")
    entry = d[key]
    x = entry.x[i]
    ys = entry.y[i]
    zs = utils.modified_zscore2(ys)
    abs_zs = abs(zs)
    for z in abs_zs:
        plt.plot(x, z, linewidth=0.5, alpha=0.5)
    plt.plot(x, utils.robust_mean2(abs_zs, tau=zval), linewidth=2.0, color='k')
    plt.hlines(0, 0, max(x), colors='k', linestyles='--')
    plt.hlines(zval, 0, max(x), colors='k', linewidth=1.0)
    plt.xlim(-10, max(x)+50)
        
    plt.minorticks_on()
    plt.tight_layout()
    plt.legend(["Modified z-score"])
    plt.xticks([])
    plt.show()

In [3]:
def spectra_scale(X, n_iter=1000, tol=1.0e-8):
    m, n = X.shape
    Y = X.mean(axis=0)
    Y /= np.sqrt(Y @ Y)

    T = X @ Y
    V = (np.outer(Y, T) - X.T)
    U = V @ T
    qval = np.sqrt((V * V).sum())
    qvals = [qval]

    for K in range(n_iter):
        qval_prev = qval
        Y = U / (U @ Y)
        Y /= np.sqrt(Y @ Y)
        T =  X @ Y

        V = (np.outer(Y, T) - X.T)
        U = V @ T
        qval = np.sqrt((V * V).sum())
        qvals.append(qval)
        if abs(qval - qval_prev) < tol: 
            break
    print(K+1, )
    return Y, T, qvals
        

In [4]:
path = f"/home/intellimath/repos/spectras/bacteria1024"
options = {}
dd = {}
widgets.load_spectras(path, dd, options)

Box(children=(VBox(children=(Label(value='дата:', style=LabelStyle(font_size='8pt', font_weight='bold')), Sele…

Button(description='Select', style=ButtonStyle())

Output()

{}

In [5]:
print(list(dd.keys()))

['SA_100_S_less100more50_6_16_1_20241021_H2O(HPLC)', 'SA_100_S_less100_6_12_1_20241017_H2O(HPLC)', 'SA_100_S_less100_6_13_1_20241017_H2O(HPLC)', 'SA_100_S_less30_6_20_1_20241024_H2O(HPLC)', 'SA_100_S_less50_6_19_1_20241022_H2O(HPLC)', 'SA_100_S_less50_6_18_1_20241022_H2O(HPLC)', 'SA_100_S_less30more10_6_15_1_20241018_H2O(HPLC)', 'SA_100_S_less50more30_6_21_1_20241024_H2O(HPLC)', 'SA_100_S_less100more50_6_17_1_20241021_H2O(HPLC)', 'SA_100_S_less50more30_6_22_1_20241024_H2O(HPLC)']


In [6]:
d_keys = list(sorted(dd.keys()))
sel_i = 0
sel = ipywidgets.Select(options=d_keys,  description="Experiment:", rows=len(dd)+1)
sel.layout.width="50%"

# def sel_on_value_change(change):
#     i_slider.value = 0
#     i_slider.max = len(dd[sel.value].y)-1

# def i_on_value_change(change):
#     plot_spectra_series(sel.value, i_slider.value, f_slider.value)

# def f_on_value_change(change):
#     plot_spectra_series(sel.value, i_slider.value, f_slider.value)

# def tau_on_value_change(change):
#     plot_spectra_series(sel.value, i_slider.value, f_slider.value, tau_slider.value)

# sel.on_trait_change(sel_on_value_change, name="value")
# i_slider.on_trait_change(i_on_value_change, name="value")
# f_slider.on_trait_change(f_on_value_change, name="value")
# tau_slider.on_trait_change(tau_on_value_change, name="value")

# ipywidgets.VBox([sel, i_slider, out])

In [7]:
for i, key in enumerate(dd):
    entry1 = dd[key]
    entry1.crop(50)
    # entry1.scale_min()
    entry1.allign_bottom()


In [ ]:
# for i, key in enumerate(dd):
#     entry1 = dd[key]
#     xs, ys = entry1.x, entry1.y
#     plt.figure(figsize=(12,4))
#     plt.title(key)
#     for x, y in zip(xs,ys):
#         mu = np.median(y, axis=0)
#         std = np.median(abs(y-mu), axis=0)
#         plt.plot(x, 100*std/mu)
#         plt.minorticks_on()
#         plt.grid(1)
#     plt.hlines(5,min(x),max(x))
#     plt.show()

In [ ]:
@ipywidgets.interact(key=sel)
def plot_spectra_for_key(key):
    entry = dd[key]
    entry.plot_spectras()

In [ ]:
# for i, key in enumerate(dd):
#     entry1 = dd[key]

    # entry1.remove_overflow_spectras(y_max=2000.0, y_max_count=30)
    # entry1.remove_outlier_spectras(delta=0.10, tau=3.5)
    # entry1.remove_by_zscore_spectras(tau=3.5, max_count=50)
    # print(len(entry1.y))

In [8]:
d2 = {}
for i, key in enumerate(dd):
    entry2 = dd[key]
    d2[key] = entry2.robust_averaging(tau=3.0)
    # entry2.smooth(tau=10.0)


In [9]:
for key in d2:
    entry = d2[key]
    entry.scale(scale=100.0)
    entry.allign_bottom()
    # entry.apply_func(y_func=np.log, x_func=np.log, b=1)
    

In [ ]:
@ipywidgets.interact(key=sel)
def plot_spectra_for_key(key, tau=2.0, ):
    entry = d2[key]
    entry.plot_spectras(tau=tau, ss=20)

In [ ]:
# X = list(d2.values())[0].y
# Y, T, qvals = spectra_scale(X)

# plt.plot(qvals)
# plt.show()
# plt.figure(figsize=(15,5))
# for y in X:
#     plt.plot(y)
# plt.plot(70*Y, color='k', linewidth=2.0)
# plt.show()
# plt.plot(sorted(T), marker='o')
# plt.show()

In [ ]:
# for key in sorted(d2.keys()):
#     entry = d2[key]
#     xs = entry.x
#     ys = entry.y
#     plt.figure(figsize=(15,4))
#     plt.title(key)
#     for x,y in zip(xs, ys):
#         plt.plot(x, y, color='k', linewidth=0.4)
#         # plt.plot(x, np.exp(at.array_modified_zscore(np.log(y+1.0)))-1.0, color='k', linewidth=0.4)
#     plt.minorticks_on()
#     plt.show()

In [ ]:
# plt.figure(figsize=(12,10))
# for key in sorted(d2.keys()):
#     entry = d2[key]
#     xs = entry.x
#     ys = entry.y
#     plt.title(key)
#     _,Ls,_ = pca.find_pc_smoothed_all(ys, 10, tau=1.0)
#     plt.plot(np.log(Ls), linewidth=2.0, marker='o', markersize=4, label=key)
# plt.minorticks_on()
# plt.legend()
# plt.show()

In [ ]:
keys = list(d2.keys())
dkeys = {}
for key in keys:
    kk = '_'.join(key.split('_')[:4])
    ll = dkeys.setdefault(kk, [])
    ll.append(key)
# print(dkeys)
# print(list(dkeys.keys()))

for ks in dkeys:
    plt.figure(figsize=(12,4))
    for kk in dkeys[ks]:
        # print(kk)
        entry = d2[kk]
        xs = entry.x
        ys = entry.y
        plt.title(ks)
        N = 25
        As,Ls,Us = pca.find_pc_smoothed_all(ys - np.mean(ys, axis=0), N, tau=10.0)
        D = np.prod(Ls) ** (1./30)
        print(D)
        entry.y /= np.sqrt(D)
        plt.plot(np.log10(Ls/D), linewidth=1.0, marker='o', markersize=4, label=kk)
        plt.minorticks_on()
        plt.legend()
    plt.show()

In [ ]:
for ks in dkeys:
    m = len(list(dkeys[ks]))
    plt.figure(figsize=(10,4))
    for kk in dkeys[ks]:
        # print(kk)
        entry = d2[kk]
        xs = entry.x
        ys = entry.y
        plt.title(ks)
        N = 25
        As,Ls,Us = pca.find_pc_smoothed_all(ys - np.mean(ys, axis=0), 2, tau=40.0)
        plt.scatter(Us[0], Us[1], label=kk)
        plt.minorticks_on()
        plt.legend()
    plt.show()

In [ ]:
keys = list(d2.keys())
dkeys = {}
for key in keys:
    kk = '_'.join(key.split('_')[:4])
    ll = dkeys.setdefault(kk, [])
    ll.append(key)
print(dkeys)
print(list(dkeys.keys()))

for ks in dkeys:
    plt.figure(figsize=(12,4))
    for kk in dkeys[ks]:
        print(kk)
        entry = d2[kk]
        xs = entry.x
        ys = entry.y
        plt.title(ks)
        # plt.plot(xs[0], utils.robust_mean2(ys, 3.0)/100, linewidth=1.0, label=kk)
        As,Ls,_ = pca.find_pc_smoothed_all(ys - np.mean(ys, axis=0), 2, tau=1.0)        
        plt.plot(xs[0], As[0], linewidth=1.0, label=kk+" / 0")
        plt.plot(xs[0], As[1], linewidth=1.0, label=kk+" / 1")
        # plt.plot(xs[0], np.median(at.array_modified_zscore2(ys), axis=0), linewidth=1.0, label=kk)
        plt.minorticks_on()
        plt.legend()
    plt.show()

In [ ]:
keys = list(d2.keys())
dkeys = {}
for key in keys:
    kk = '_'.join(key.split('_')[:4])
    ll = dkeys.setdefault(kk, [])
    ll.append(key)
# print(dkeys)
# print(list(dkeys.keys()))

wma = averaging_function("WMZ")

for ks in dkeys:
    plt.figure(figsize=(12,4))
    for kk in dkeys[ks]:
        # print(kk)
        entry = d2[kk]
        xs = entry.x
        ys = entry.y
        plt.title(ks)
        _,Ls,_ = pca.find_robust_pc_all(ys, wma, 10)
        plt.plot(np.log(Ls), linewidth=1.0, marker='o', markersize=4, label=kk)
        plt.minorticks_on()
        plt.legend()
    plt.show()

In [10]:
def func1(E):
    # return (1 - E / np.sqrt(1 + E*E)) / 2
    return special.expit(-10*(E-4))

# def func2(E,D):    
#     return (1 + E / np.sqrt(1 + E*E))
#     E = abs(E)
#     return E / E.max() 

@ipywidgets.interact(key=sel)
def plot_spectra_for_key(key):
    entry = d2[key]
    entry.select_baselines(tau2=1.0e4, tau1=0.0, bs_scale=3.0, solver="fast", d=2, func2_mode="d",
                           func=funcs.Expit(-10.0, -2.0).evaluate_array,
                           func1=None, #funcs.().evaluate,
                           func2=None, #funcs.Expit(10.0, 0.5),
                          )

interactive(children=(Select(description='Experiment:', layout=Layout(width='50%'), options=('SA_100_S_less100…

In [ ]:
for i, key in enumerate(d2):
    entry3 = d2[key]
    # entry3.smooth(method="irsa", tau=1.)
    # entry3.allign_bottom()
    # entry3.normalize_area_under_curve()
    # entry3.remove_by_zscore_spectras(tau=3.5, max_count=40)
    entry3.subtract_baseline("irsa", lam=100.0)
    # entry3.replace_small_values(0.01)
    # entry3.normalize_area_under_curve()
    # print(entry3.x.shape, entry3.y.shape)
    # entry3.y = entry3.remove_outlier_spectras(tau=3.5)
    # Is = [i for i,ys in enumerate(entry3.y) if np.any(np.isnan(ys))]
    # print(Is)            


In [ ]:
for i, key in enumerate(d2):
    entry = d2[key]
    Xs, Ys = entry.x, entry.y

    plt.figure(figsize=(12,4))
    plt.title(key)
    for xs, ys in zip(Xs, Ys):
        plt.plot(xs, ys, linewidth=0.5)
    plt.hlines(0, min(xs), max(xs), colors='k', linestyles='--')
    plt.xlim(min(xs)-20, max(xs))
        
    plt.minorticks_on()
    plt.tight_layout()
    # plt.legend()
    plt.show()
    
    # plt.figure(figsize=(12,3))
    # Zs = utils.modified_zscore2(Ys)
    # for xs,zs in zip(Xs,Zs):
    #     plt.plot(xs,abs(zs), linewidth=0.5, marker='s', markersize=1)
    # plt.hlines(3.5, 0, max(xs), colors='k', linestyle='--')
    # plt.minorticks_on()
    # plt.tight_layout()
    # plt.show()
    

In [ ]:
from sklearn.decomposition import PCA
from numpy.linalg import det, inv

plt.figure(figsize=(10,6))
for i, key in enumerate(d2):
    entry = d2[key]
    Xs, Ys = entry.x, entry.y

    pca = PCA(5)
    pca.fit(Ys)
    Us = pca.transform(Ys)

    S = Us.T @ Us
    S /= np.sqrt(det(S))
    S = inv(S)
    # print(S)


    sc = plt.scatter(Us[:,0], Us[:,1], label=key)
    c = sc.get_facecolor()

    # x_min, x_max = min(Us[:,0]), max(Us[:,0])
    # y_min, y_max = min(Us[:,1]), max(Us[:,1])

    # xr = np.linspace(x_min, x_max, 50)
    # yr = np.linspace(y_min, y_max, 50)
    # xx, yy = np.meshgrid(xr, yr)
    # xy = np.c_[xx.ravel(), yy.ravel()]
    # dd = np.sqrt([0.5*((t @ S) @ t) for t in xy])
    # dd = dd.reshape(xx.shape)
    # plt.contour(dd, levels=[4.0], colors=c)
    

plt.minorticks_on()
plt.tight_layout()
plt.legend()
plt.show()

In [ ]:
from mlgrad.pca import location, find_pc_all, find_pc_l1_all

plt.figure(figsize=(10,6))
for i, key in enumerate(d2):
    entry = d2[key]
    Xs, Ys = entry.x, entry.y

    C = location(Ys)
    As, Ls, Us = find_pc_all(Ys - C, 2)
    plt.plot(Ls, marker='o', label=key)
    # plt.scatter(Us[0], Us[1], label=key)

plt.tight_layout()
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
for i, key in enumerate(d2):
    entry = d2[key]
    Xs, Ys = entry.x, entry.y

    # if "_R_" in key:
    #     if "95" in key:
    #         color='r'
    #     elif "96" in key:
    #         color='Brown'
    # elif "_S_" in key:
    #     if "92" in key:
    #         color='m'
    #     elif "93" in key:
    #         color='b'
    #     else:
    #         color='b'

    if "_R_" in key:
        color='r'
    elif "_S_" in key:
        color='b'
    else:
        color='m'
        
    # plt.plot(np.mean(Xs, axis=0), np.mean(Ys, axis=0), linewidth=1.0, color=color, label=key)
    plt.plot(np.mean(Ys, axis=0), linewidth=1.0, label=key)
    
# plt.hlines(0, 0, len(Ys[0]), colors='k')
# plt.xlim(-10,1000)
        
plt.minorticks_on()
plt.tight_layout()
plt.legend()
plt.show()

In [2]:
import sympy
from sympy import MatrixSymbol, Identity, latex

In [3]:
#D = MatrixSymbol('D', 10, 10)
# W = sympy.DiagonalMatrix(MatrixSymbol('w', 8, 8))
w = sympy.symarray("w", (8,), )
W = np.diag(w[1:-1])

In [4]:
E = np.eye(8, dtype="i")
D2 = np.diff(E, 2, axis=0)
print(D2)
print(D2.T @ D2)

[[ 1 -2  1  0  0  0  0  0]
 [ 0  1 -2  1  0  0  0  0]
 [ 0  0  1 -2  1  0  0  0]
 [ 0  0  0  1 -2  1  0  0]
 [ 0  0  0  0  1 -2  1  0]
 [ 0  0  0  0  0  1 -2  1]]
[[ 1 -2  1  0  0  0  0  0]
 [-2  5 -4  1  0  0  0  0]
 [ 1 -4  6 -4  1  0  0  0]
 [ 0  1 -4  6 -4  1  0  0]
 [ 0  0  1 -4  6 -4  1  0]
 [ 0  0  0  1 -4  6 -4  1]
 [ 0  0  0  0  1 -4  5 -2]
 [ 0  0  0  0  0  1 -2  1]]


In [5]:
DD2 = np.dot(np.dot(D2.T, W), D2)

In [6]:
from IPython.display import display, Latex

In [7]:
s = r"$$\left(\begin{array}{llllllll}"
for i in range(8):
    for j in range(8):
        s += latex(DD2[i,j])
        if j < 7:
            s += r"&"
        else:
            s += r"\\"
s += r"\end{array}\right)$$"
# print(s)
display(Latex(s))

<IPython.core.display.Latex object>

In [8]:
D1 = np.diff(np.eye(8), 1, axis=0, )
print(D1)
print(D1.T @ D1)

[[-1.  1.  0.  0.  0.  0.  0.  0.]
 [ 0. -1.  1.  0.  0.  0.  0.  0.]
 [ 0.  0. -1.  1.  0.  0.  0.  0.]
 [ 0.  0.  0. -1.  1.  0.  0.  0.]
 [ 0.  0.  0.  0. -1.  1.  0.  0.]
 [ 0.  0.  0.  0.  0. -1.  1.  0.]
 [ 0.  0.  0.  0.  0.  0. -1.  1.]]
[[ 1. -1.  0.  0.  0.  0.  0.  0.]
 [-1.  2. -1.  0.  0.  0.  0.  0.]
 [ 0. -1.  2. -1.  0.  0.  0.  0.]
 [ 0.  0. -1.  2. -1.  0.  0.  0.]
 [ 0.  0.  0. -1.  2. -1.  0.  0.]
 [ 0.  0.  0.  0. -1.  2. -1.  0.]
 [ 0.  0.  0.  0.  0. -1.  2. -1.]
 [ 0.  0.  0.  0.  0.  0. -1.  1.]]


In [9]:
w = sympy.symarray("w", (7,))
W = np.diag(w)
DD1 = np.dot(np.dot(D1.T, W), D1)

In [10]:
s = r"$$\left(\begin{array}{llllllll}"
for i in range(8):
    for j in range(8):
        s += latex(DD1[i,j])
        if j < 7:
            s += r"&"
        else:
            s += r"\\"
s += r"\end{array}\right)$$"
# print(s)
display(Latex(s))

<IPython.core.display.Latex object>

In [ ]:
D3 = np.diff(np.eye(9), 3, axis=0, )
print(D3)
print(D3.T @ D3)

In [ ]:
import ipywidgets as widgets

In [ ]:
buttons = []
for i in range(10):
    buttons.append(widgets.Button(description=f"  The button {i}  "))
box = ipywidgets.Box(buttons)
box.layout = widgets.Layout(flex_flow="row wrap")
box

In [ ]:
plt.fill_between()